In [4]:
from manim import * 

In [28]:
%%manim -ql -v WARNING LagInterp
from functools import partial
class LagInterp(MovingCameraScene):
    def clean(self):
        #FadeOut a todo en pantalla
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )
    def construct(self):
        title = Tex("Interpolación de Lagrange")
        self.play(Write(title))
        self.wait(2)
        self.play(Unwrite(title))
        source = Tex(r"source:https://github.com/TheRookieNerd/ManimProjects2022/blob/main/LagInterp.py").scale(0.4).to_edge(DOWN)
        self.play(Write(source))
        # self.camera.frame.scale(1.5)
        ax = Axes(
            x_range=[-1, 7, 1],
            y_range=[-18, 10, 2],
            y_length=14,
            x_axis_config={
                "include_tip": False
            },
            y_axis_config={
                "include_tip": False
            }
        ).add_coordinates().shift(DOWN * 4 + LEFT * 0.75)
        # ax = NumberPlane().add_coordinates()
        self.play(Create(ax))
        self.wait()
        colors_list = [YELLOW, GREEN, BLUE, PINK]

        ip_pts = [
            (0, 3), (1, 5), (3.5, 1), (5, 3)
        ]
        scale_fac = 0.65
        eqns1 = VGroup(
            MathTex("f(x) =").scale(1 / scale_fac),
            MathTex("3\\,\\dfrac{(x-1)(x-3.5)(x-5)}{(0-1)(0-3.5)(0-5)}", color=colors_list[0]),
            MathTex("+"),
            MathTex("5\\,\\dfrac{(x-0)(x-3.5)(x-5)}{(1-0)(1-3.5)(1-5)}", color=colors_list[1]),
            MathTex("+"),
        ).scale(scale_fac).arrange_submobjects(direction=RIGHT).to_edge(UP, buff=1)
        eqns2 = VGroup(
            MathTex("1\\,\\dfrac{(x-0)(x-1)(x-5)}{(3.5-0)(3.5-1)(3.5-5)}", color=colors_list[2]),
            MathTex("+"),
            MathTex("3\\,\\dfrac{(x-0)(x-1)(x-3.5)}{(5-0)(5-1)(5-3.5)}", color=colors_list[3])
        ).scale(scale_fac).arrange_submobjects(direction=RIGHT).next_to(eqns1[3], direction=DOWN)
        f1 = MathTex("f_1 = ").scale(0.75).next_to(eqns1[1], direction=LEFT, buff=0.1)
        eqns = VGroup(eqns1, eqns2).add_background_rectangle()
        
        pts = [ax.c2p(pt[0], pt[1]) for pt in ip_pts]
        # pts = [RIGHT * pt[0] + UP * pt[1] for pt in [
        #     (0, 2), (1, 1), (1.5, 4), (3, 0.5)
        # ]]
        dots = VGroup()

        for pt, c in zip(pts, colors_list):
            dot = Dot(pt, color=c).scale(0.75)
            dot.add(Circle(radius=0.1, stroke_width=1, color=c).move_to(dot))
            dots.add(dot)

        def demolagfn1(x):
            nr = 1
            for pt2 in ip_pts:
                if not np.array_equal(ip_pts[0], pt2):
                    nr *= x - pt2[0]
            return nr

        def demolagfn2(x):
            nr, dr = 1, 1
            for pt2 in ip_pts:
                if not np.array_equal(ip_pts[0], pt2):
                    nr *= x - pt2[0]
                    dr *= ip_pts[0][0] - pt2[0]
            return (nr / dr)

        def nthlagfn(x, pt):
            nr, dr = 1, 1
            for pt2 in ip_pts:
                if not np.array_equal(pt, pt2):
                    nr *= x - pt2[0]
                    dr *= pt[0] - pt2[0]
            return pt[1] * (nr / dr)

        functions = {f'fn{i}': partial(nthlagfn, pt=k) for i, k in enumerate(ip_pts)}
        l_plots = VGroup()
        for i, c in enumerate(colors_list):
            l_plots.add(ax.plot(functions[f"fn{i}"], x_range=[-0.1, 5.5], color=c))

        def lagfn(x):
            # self.ls=[]
            l = 0
            for pt in ip_pts:
                nr, dr = 1, 1
                for pt2 in ip_pts:
                    if not np.array_equal(pt, pt2):
                        nr *= x - pt2[0]
                        dr *= pt[0] - pt2[0]

                l += pt[1] * (nr / dr)
            return l

        rev_colors_list = colors_list[-1::-1]
        interp = ax.plot(lagfn, x_range=[-0.5, 5.5]).set_color(color=rev_colors_list)

        scale_fac = 0.2
        l_plots_cpy = VGroup()
        for i, l in enumerate(l_plots.copy()):
            l_plots_cpy.add(l)
            if i != 3:
                l_plots_cpy.add(MathTex("+").scale(3))

        l_plots_cpy.add(MathTex("=").scale(3), interp.copy())
        l_plots_cpy.arrange_submobjects(buff=0.75).to_edge(UP).scale(scale_fac).shift(RIGHT * 0.25)
        # self.add(l_plots_cpy)

        coords = VGroup(*[MathTex(pt) for pt in ip_pts])
        vert_lines = VGroup(*[ax.get_vertical_line(ax.c2p(pt[0], pt[1]), color=WHITE, stroke_width=3).rotate(180 * DEGREES) for pt in ip_pts[1:]])
        circ = VGroup(*[Circle(radius=0.1, stroke_width=2, color=WHITE).move_to(vert_lines[_].get_end()) for _ in range(3)])

        for dot, coord in zip(dots, coords):
            coord.next_to(dot)
            self.play(FadeIn(dot), Write(coord))
        self.wait()
        self.play(FadeOut(coords, title))

        l_plots.save_state()
        for i, l in enumerate(l_plots):
            self.play(
                Create(l),
                run_time=2
            )
            self.wait()
            if i == 0:
                ddots = VGroup(*[Dot(vert_lines[_].get_end()).scale(0.1) for _ in range(3)])
                self.play(*[Flash(ddot) for ddot in ddots])
            self.play(l.animate.fade(0.8))

        self.play(*[ReplacementTransform(i, j) for i, j in zip(l_plots, l_plots_cpy[0:-2:2])])
        self.play(FadeIn(l_plots_cpy[1::2], l_plots_cpy[-1]))
        self.wait()
        self.play(FadeOut(l_plots_cpy))
        l_plots.restore()

        demo_fn1 = ax.plot(demolagfn1, x_range=[0, 5.5], color=YELLOW_C)
        demo_fn2 = ax.plot(demolagfn2, color=YELLOW_D)
        self.camera.frame.save_state()

        self.play(Create(demo_fn1), Write(f1), Write(eqns1[1][0][1:18]), run_time=2)

        self.play(self.camera.frame.animate.shift(DOWN * 9))
        intercept = ax.i2gp(0, demo_fn1)
        c = DecimalNumber(demolagfn1(0)).next_to(intercept)
        self.play(Write(c))

        self.play(
            self.camera.frame.animate.restore(),
            c.animate.next_to(eqns1[1][0][18], direction=DOWN, buff=0.1).scale(0.65).set_color(YELLOW),
            Write(eqns1[1][0][18]),
            ReplacementTransform(demo_fn1, demo_fn2),
            run_time=3
        )
        temp = VGroup(
            MathTex("1").scale(0.75).next_to(ax.c2p(0, 1), direction=LEFT, buff=0.2),
            MathTex("3").scale(0.75).next_to(ax.c2p(0, 3), direction=LEFT, buff=0.2)
        )
        self.play(FadeIn(temp[0]))
        self.play(Flash(temp[0]))
        self.wait()
        self.play(ReplacementTransform(c, eqns1[1][0][19:]))
        self.wait()

        self.play(
            ReplacementTransform(demo_fn2, l_plots[0]),
            Write(eqns1[1][0][0]),
            ReplacementTransform(temp[0], temp[1]),
            run_time=2
        )

        self.wait()
        self.play(
            ShowPassingFlash(
                l_plots[0].copy().set_color(RED),
                run_time=2,
                time_width=0.5
            )
        )
        self.play(FadeOut(temp[1]))

        for l, c in zip(vert_lines, circ):
            self.play(Create(l), Create(c), run_time=0.5)
        self.wait()
        self.play(FadeOut(circ, vert_lines))
        self.wait()

        self.play(l_plots[0].animate.fade(0.95), FadeOut(f1))
        for l, eqn in zip(l_plots[1:], [eqns1[-2], eqns2[0], eqns2[-1]]):
            self.play(
                Create(l),
                Write(eqn),
                run_time=2
            )
            self.wait()
            self.play(l.animate.fade(0.8))
        # self.add(l_plots[1:], interp)
        # l_plots.fade(0.8)
        self.play(
            Create(interp),
            *[Write(mob) for mob in [eqns1[0], eqns1[2], eqns1[-1], eqns2[1]]]
        )
        self.wait()
        self.play(*[FadeOut(mobj) for mobj in self.mobjects])
        

        scale_fac = 0.45


        
        
        # Definir las partes de la fórmula
        prefix = MathTex(r"p(x) = \sum_{i=1}^{n}y_i")
        term = MathTex(r"L_i(x)")

        # Alinear las partes de la fórmula
        formula = VGroup(prefix, term)
        formula.arrange(RIGHT)
       
        transformed_term = MathTex(r"\frac{l_i(x)}{l_i(x_i)}").move_to(term)
        numerator, denominator = transformed_term[0][:5], transformed_term[0][6:12]
        # Crear la animación
        self.play(Write(formula))
        self.wait(2)

        # Transformar el término
        self.play(ReplacementTransform(term, transformed_term))
        
        self.wait(2)

        formula.add(transformed_term)
        
        # Mover la formula al borde izquierdo
        self.play(ApplyMethod(formula.to_edge, LEFT))
        self.wait(2)

        # Destacar el li/li
        self.play(denominator.animate.set_color(GREEN), numerator.animate.set_color(RED))
        self.wait(2)


        
        # Crear una copia del numerador y moverlo al centro de la pantalla
        numerator_copy = numerator.copy().move_to(ORIGIN + UP*2.5 + LEFT)
        self.play(ReplacementTransform( numerator.copy(), numerator_copy))
        self.wait(2)
        product_numerator = MathTex(r"= \prod_{k=1,i\neq k}^{n}(x - x_k)").scale(0.7).next_to(numerator_copy,RIGHT)
        self.play(Write(product_numerator))
        text_numerator = Text("Se debe evaluar cada vez que evaluemos el polinomio\ninterpolador p(x)", font_size = 18).move_to(product_numerator.get_right()+DOWN + LEFT*0.5)
        
        self.play(Write(text_numerator))
        self.play(Circumscribe(text_numerator[6:20],),text_numerator[6:20].animate.set_color(RED))
        
        
        # Crear una copia del denominador y moverlo al centro de la pantalla
        denominator_copy = denominator.copy().move_to(ORIGIN + DOWN + LEFT)
        self.play(ReplacementTransform(denominator.copy(), denominator_copy))
        self.play(denominator_copy.animate)
        self.wait(2)
        
        product = MathTex(r"= \prod_{k=1,i\neq k}^{n}(x_i - x_k)").scale(0.7).next_to(denominator_copy,RIGHT)
        self.play(Write(product))
        self.play(Indicate(denominator_copy[3:5]),Indicate(product[0][12:14],1.5))
        self.wait(2)
        
        text_denominator = Text("Al depender exclusivamente de los puntos interpoladores \nse puede pre-calcular.", font_size=18).move_to(product.get_right()+DOWN+LEFT*0.5)
        
        self.play(Write(text_denominator))
        # Destacado a la mala
        self.play(Circumscribe(text_denominator[56:67]),text_denominator[56:68].animate.set_color(GREEN))
        self.wait(2)
        
        self.wait(5)
        group = VGroup(denominator_copy,product,text_denominator,numerator_copy,product_numerator,text_numerator)
        self.play(Unwrite(group), Unwrite(formula))
        
        # END
        
        # START 2nd scene
        
        colors_list = [YELLOW, GREEN, BLUE, PINK]

        
        #FadeOut a todo en pantalla
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )
        eqns = VGroup(
        MathTex("p(x) ="),
        MathTex("3\\,\\dfrac{(x-1)(x-3.5)(x-5)}{(0-1)(0-3.5)(0-5)}", color=colors_list[0]),
        MathTex("+"),
        MathTex("5\\,\\dfrac{(x-0)(x-3.5)(x-5)}{(1-0)(1-3.5)(1-5)}", color=colors_list[1]),
        MathTex("+"),
        MathTex("1\\,\\dfrac{(x-0)(x-1)(x-5)}{(3.5-0)(3.5-1)(3.5-5)}", color=colors_list[2]),
        MathTex("+"),
        MathTex("3\\,\\dfrac{(x-0)(x-1)(x-3.5)}{(5-0)(5-1)(5-3.5)}", color=colors_list[3])
        ).scale(scale_fac).arrange_submobjects(direction=RIGHT).to_edge(UP, buff=1)
        self.play(Write(eqns))
        op_elem_precal = Tex("Operaciones elementales para pre-calcular $l_i(x_i)$:").scale(0.7).move_to(LEFT*2.5+UP*1.5)
        
        tex_restas = Tex("$n-1$ sustracciones").scale(0.7) 
        tex_multi = Tex("$n-2$ multiplicaciones").scale(0.7)
       
        tex_total = VGroup(
            Tex("Dado n puntos:").scale(0.7),
            Tex("$n(2n-3)$").scale(0.7),
            Tex("para pre-calcular").scale(0.7),
            MathTex(r"l_i(x_i)").scale(0.7)
        ).arrange(RIGHT)
        tex_op_elementales = VGroup(op_elem_precal, tex_restas, tex_multi, tex_total).arrange(DOWN, center=False, aligned_edge=LEFT)
        to_transform1 = Tex("$2n^2-3n$").scale(0.7).move_to(tex_total[1])
        to_transform2 = Tex("$O(n^2)$").scale(0.7).move_to(tex_total[1])
        
        self.play(
            Write(op_elem_precal)
        )
        self.play(Indicate(eqns[1][0][19:37]))
        self.play(Indicate(eqns[3][0][19:37]))
        self.play(Indicate(eqns[5][0][17:38]))
        self.play(Indicate(eqns[7][0][19:37]))
        self.play(Write(tex_restas))
        #ANIMAR RESTAS
        self.play(
            *[Indicate(eqns[idx][0][21], scale_factor = 2.0) for idx in [1,3,7]],
            *[Indicate(eqns[idx][0][26], scale_factor = 2.0) for idx in [1,3,7]],
            *[Indicate(eqns[idx][0][33], scale_factor = 2.0) for idx in [1,3]],
            *[Indicate(eqns[5][0][idx], scale_factor = 2.0) for idx in [21,28,35]],
            *[Indicate(eqns[7][0][idx], scale_factor = 2.0) for idx in [21,26,31]],
            
        )
        self.play(
            Write(tex_multi)
        )
        
        #ANIMAR MULTIPLICACIONES
        self.play(
            *[Indicate(eqns[idx][0][19:37]) for idx in [1,3]],
            Indicate(eqns[5][0][17:38]), 
            Indicate(eqns[7][0][19:37])
            
        )
        self.play(Write(tex_total[0]))
        self.play(Write(tex_total[1:]))
        self.wait(1)
        self.play(ReplacementTransform(tex_total[1],to_transform1))
        self.wait(1)
        self.play(ReplacementTransform(to_transform1,to_transform2))

        self.wait(3)
        
        colors_list = [YELLOW, GREEN, BLUE, PINK]

        # PRIMERO SCALE
        self.clean()
        eqns1 = VGroup(
            MathTex("p(x) ="),
            MathTex("3\\,\\dfrac{(x-1)(x-3.5)(x-5)}{-17.5}", color=colors_list[0]),
            MathTex("+"),
            MathTex("5\\,\\dfrac{(x-0)(x-3.5)(x-5)}{10}", color=colors_list[1]),
            MathTex("+"),
            MathTex("1\\,\\dfrac{(x-0)(x-1)(x-5)}{-13.125}", color=colors_list[2]),
            MathTex("+"),
            MathTex("3\\,\\dfrac{(x-0)(x-1)(x-3.5)}{30}", color=colors_list[3])
        ).scale(scale_fac).arrange_submobjects(direction=RIGHT).to_edge(UP, buff=1)
        self.play(Write(eqns1))
        l_group = VGroup(
            Tex(r"$L_1(x)$").scale(0.5).next_to(eqns1[1], UP),
            Tex(r"$L_2(x)$").scale(0.5).next_to(eqns1[3], UP),
            Tex(r"$L_3(x)$").scale(0.5).next_to(eqns1[5], UP),
            Tex(r"$L_4(x)$").scale(0.5).next_to(eqns1[7], UP),
        )
        buff_bullet = 0.9
        tex_title = Tex("Operaciones elementales para evaluar $p(x)$:").scale(0.8).move_to(LEFT*3+UP*1.5)
        tex_li = Tex("Para un solo $L_i(x)$:").scale(0.7).next_to(tex_title, DOWN, aligned_edge=LEFT, buff = buff_bullet)
        tex_restas = Tex("$n-1$ sustracciones").scale(0.7).next_to(tex_li, RIGHT)
        tex_multi = Tex("$n-2$ multiplicaciones").scale(0.7)#.next_to(tex_restas, DOWN)
        tex_div = Tex("1 division").scale(0.7)
        tex_op_to_transform = Tex("$2n-2$").next_to(tex_li, RIGHT).scale(0.7)
        tex_xtimesL = Tex("Multiplicar $x_i$ por $L_i(x)$:").scale(0.7).next_to(tex_li, DOWN, aligned_edge = LEFT, buff = buff_bullet)
        tex_op_elementales = VGroup(tex_restas, tex_multi, tex_div).arrange(DOWN, center=False, aligned_edge = LEFT)

        

        
        
        #TODO - Tener operaciones elementales a un lado y por el otro ir acumuladolas
        
        self.play(Write(tex_title))
        self.play(Write(l_group),Write(tex_li))
        self.play(Write(tex_restas))
        self.play(
            *[Indicate(eqns1[idx][0][3], scale_factor = 2.0) for idx in [1,3,5,7]],
            *[Indicate(eqns1[idx][0][8], scale_factor = 2.0) for idx in [1,3,5,7]],
            *[Indicate(eqns1[idx][0][15], scale_factor = 2.0) for idx in [1,3]], 
            *[Indicate(eqns1[idx][0][13], scale_factor = 2.0) for idx in [5,7]]
        )
        self.play(Write(tex_multi))
        self.play(
            *[Indicate(eqns1[idx][0][1:18]) for idx in [1,3,7]],
            Indicate(eqns1[5][0][1:16])
        )
        self.play(Write(tex_div))
        self.play(
            *[Indicate(eqns1[idx][0][1:]) for idx in [1,3,5,7]]
        )



        self.play(ReplacementTransform(tex_op_elementales,tex_op_to_transform))
        self.play(Write(tex_xtimesL))
        self.play(Indicate(eqns1[1]))
        self.play(tex_op_to_transform.animate.next_to(tex_xtimesL, RIGHT))
        #self.play(ApplyMethod(tex_op_to_transform.move_to,tex_xtimesL,RIGHT, aligned_edge = LEFT))
        
        tex_plus = Tex("+ 1").scale(0.7).next_to(tex_op_to_transform, RIGHT)
        tex_group = VGroup(tex_op_to_transform, tex_plus)
        tex_op_to_transform2 = Tex("$2n-1$").scale(0.7).move_to(tex_group, aligned_edge = LEFT)

        
        tex_total = Tex("Considerando los n terminos de p(x):").scale(0.7).next_to(tex_xtimesL, DOWN, aligned_edge = LEFT, buff = buff_bullet)
        
        tex_op_to_transform3 = Tex("$n(2n-1)$").scale(0.7).next_to(tex_total, RIGHT)
        tex_op_to_transform4 = Tex("$2n^2-n$").scale(0.7).next_to(tex_total, RIGHT)
        self.play(Write(tex_plus))
        tex_op_to_transform5 = Tex("$O(n^2)$").scale(0.7).move_to(tex_op_to_transform4, aligned_edge = LEFT)        
        self.play(ReplacementTransform(tex_group,tex_op_to_transform2))
        self.play(Write(tex_total))
        for i in [1,3,5,7]:
            self.play(Indicate(eqns1[i]))
        self.play(tex_op_to_transform2.animate.next_to(tex_total, RIGHT))
        self.play(ReplacementTransform(tex_op_to_transform2,tex_op_to_transform3 ))
        self.wait(1)
        self.play(ReplacementTransform(tex_op_to_transform3,tex_op_to_transform4 ))
        self.wait(1)       
        self.play(ReplacementTransform(tex_op_to_transform4,tex_op_to_transform5 ))
        self.wait(1)
        
        #conclusion
        self.play(Circumscribe(tex_title[0][26:38]),Circumscribe(tex_op_to_transform5))
        
        self.clean()
        
#class LagrangeDefinitivo(Scene):

        
        
    
        

In [10]:
%%manim -ql -v WARNING Baricentrica
from manim import *
from functools import partial
import numpy as np


class Baricentrica(MovingCameraScene):
    def construct(self):

        # self.camera.frame.scale(1.5)
        ax = Axes(
            x_range=[-1, 7, 1],
            y_range=[-18, 8, 2],
            y_length=10,
            x_axis_config={
                "include_tip": False
            },
            y_axis_config={
                "include_tip": False
            }
        ).add_coordinates().shift(DOWN * 4 + LEFT * 0.75)
        # ax = NumberPlane().add_coordinates()
        self.play(Create(ax))
        self.wait()
        colors_list = [YELLOW, GREEN, BLUE, PINK]

        ip_pts = [
            (0, 3), (1, 5), (3.5, 1), (5, 3)
        ]
        scale_fac = 0.65
        eqns1 = VGroup(
            MathTex("p(x) =").scale(1 / 0.9),
            MathTex("\dfrac{\dfrac{w_1*3}{x-0}}{\dfrac{w_1}{x-0}+\dfrac{w_2}{x-1}+\dfrac{w_3}{x-3.5}+\dfrac{w_4}{x-5}}", color=colors_list[0]).scale(0.9),
            MathTex("+"),
            MathTex("\dfrac{\dfrac{w_2*5}{x-1}}{\dfrac{w_1}{x-0}+\dfrac{w_2}{x-1}+\dfrac{w_3}{x-3.5}+\dfrac{w_4}{x-5}}", color=colors_list[1]).scale(0.9),
            MathTex("+"),
        ).scale(scale_fac).arrange_submobjects(direction=RIGHT).to_edge(UP, buff=0.5)
        eqns2 = VGroup(
            MathTex("\dfrac{\dfrac{w_3*1}{x-3.5}}{\dfrac{w_1}{x-0}+\dfrac{w_2}{x-1}+\dfrac{w_3}{x-3.5}+\dfrac{w_4}{x-5}}", color=colors_list[2]).scale(0.9),
            MathTex("+"),
            MathTex("\dfrac{\dfrac{w_4*3}{x-5}}{\dfrac{w_1}{x-0}+\dfrac{w_2}{x-1}+\dfrac{w_3}{x-3.5}+\dfrac{w_4}{x-5}}", color=colors_list[3]).scale(0.9)
        ).scale(scale_fac).arrange_submobjects(direction=RIGHT).next_to(eqns1[2], direction=DOWN, buff = 0.6)
        f1 = MathTex("f_1 = ").scale(0.75).next_to(eqns1[1], direction=LEFT, buff=0.1)
        eqns = VGroup(eqns1, eqns2).add_background_rectangle()
        eqn_condensed = MathTex(
            "p(x) = \dfrac{\dfrac{w_1*3}{x-0}+\dfrac{w_2*5}{x-1}+\dfrac{w_3*1}{x-3.5} + \dfrac{w_4*3}{x-5}}{\dfrac{w_1}{x-0}+\dfrac{w_2}{x-1}+\dfrac{w_3}{x-3.5}+\dfrac{w_4}{x-5}}"
        ).scale(scale_fac).move_to(eqns)
        pts = [ax.c2p(pt[0], pt[1]) for pt in ip_pts]

        dots = VGroup()

        for pt, c in zip(pts, colors_list):
            dot = Dot(pt, color=c).scale(0.75)
            dot.add(Circle(radius=0.1, stroke_width=1, color=c).move_to(dot))
            dots.add(dot)
            
        def demolagfn1(x):
            w = 1/np.array([
                (0-1)*(0-3.5)*(0-5), 
                 (1-0)*(1-3.5)*(1-5),
                (3.5)*(3.5-1)*(3.5-5),
                (5)*(5-1)*(5-3.5)
                ])
            if np.isclose(x,0):
                return 3
            nr = w[0]*3/x
            
            return nr

        def demolagfn2(x):
            w = 1/np.array([
                (0-1)*(0-3.5)*(0-5), 
                 (1-0)*(1-3.5)*(1-5),
                (3.5)*(3.5-1)*(3.5-5),
                (5)*(5-1)*(5-3.5)
                ])
            if np.isclose(x,0):
                return 3
            nr = w[0]*3/x
            dr = sum(w[idx] / (x - pt[0]) for idx, pt in enumerate(ip_pts))
            return nr/dr

        def nthlagfn(x, pt, ind):
            w = 1/np.array([
                (0-1)*(0-3.5)*(0-5), 
                 (1-0)*(1-3.5)*(1-5),
                (3.5)*(3.5-1)*(3.5-5),
                (5)*(5-1)*(5-3.5)
                ])
            nr, dr = 0, 0
            if np.isclose(x, pt[0]):
                return pt[1]

            dr = sum(w[idx] / (x - pt[0]) for idx, pt in enumerate(ip_pts))
            nr = (w[ind] * pt[1]) / (x - pt[0])
    
            return nr / dr

        functions = {f'fn{i}': partial(nthlagfn, pt=k, ind=i) for i, k in enumerate(ip_pts)}
        l_plots = VGroup()
        for i, c in enumerate(colors_list):
            l_plots.add(ax.plot(functions[f"fn{i}"], x_range=[-0.1, 5.5], color=c))

        def lagfn(x):
            w = 1/np.array([(0-1)*(0-3.5)*(0-5),(1-0)*(1-3.5)*(1-5),(3.5)*(3.5-1)*(3.5-5),(5)*(5-1)*(5-3.5)])
            # self.ls=[]
            l = 0
            nr = 0
            dr = 0
            
            for idx,pt in enumerate(ip_pts):
                if x == pt[0]:
                    return pt[1]
                
                nr += (w[idx]*pt[1])/(x-pt[0])
                dr += w[idx]/(x-pt[0])
            l = nr/dr
            return l

        rev_colors_list = colors_list[-1::-1]
        interp = ax.plot(lagfn, x_range=[-0.5, 5.5]).set_color(color=rev_colors_list)

        scale_fac = 0.2
        l_plots_cpy = VGroup()
        for i, l in enumerate(l_plots.copy()):
            l_plots_cpy.add(l)
            if i != 3:
                l_plots_cpy.add(MathTex("+").scale(3))

        l_plots_cpy.add(MathTex("=").scale(3), interp.copy())
        l_plots_cpy.arrange_submobjects(buff=0.75).to_edge(UP).scale(scale_fac).shift(RIGHT * 0.25)
        # self.add(l_plots_cpy)

        coords = VGroup(*[MathTex(pt) for pt in ip_pts])
        vert_lines = VGroup(*[ax.get_vertical_line(ax.c2p(pt[0], pt[1]), color=WHITE, stroke_width=3).rotate(180 * DEGREES) for pt in ip_pts[1:]])
        circ = VGroup(*[Circle(radius=0.1, stroke_width=2, color=WHITE).move_to(vert_lines[_].get_end()) for _ in range(3)])

        for dot, coord in zip(dots, coords):
            coord.next_to(dot)
            self.play(FadeIn(dot), Write(coord))
        self.wait()
        self.play(FadeOut(coords))

        l_plots.save_state()
        for i, l in enumerate(l_plots):
            self.play(
                Create(l),
                run_time=2
            )
            self.wait()
            if i == 0:
                ddots = VGroup(*[Dot(vert_lines[_].get_end()).scale(0.1) for _ in range(3)])
                self.play(*[Flash(ddot) for ddot in ddots])
            self.play(l.animate.fade(0.8))

        self.play(*[ReplacementTransform(i, j) for i, j in zip(l_plots, l_plots_cpy[0:-2:2])])
        self.play(FadeIn(l_plots_cpy[1::2], l_plots_cpy[-1]))
        self.wait()
        self.play(FadeOut(l_plots_cpy))
        l_plots.restore()

        demo_fn1 = ax.plot(demolagfn1, color=YELLOW_C, x_range=[0.099, 5.5])
        demo_fn2 = ax.plot(demolagfn2, color=YELLOW_D, x_range=[-0.5,7])
        self.camera.frame.save_state()
        self.play(Write(eqns1[1][0][0:8]))
        self.play(Write(demo_fn1))

        #self.play(self.camera.frame.animate.shift(DOWN * 9))
        #intercept = ax.i2gp(0, demo_fn1)
        #c = DecimalNumber(demolagfn1(0)).next_to(intercept)
        #self.play(Write(c))

       # self.play(
        #    self.camera.frame.animate.restore(),
        #    c.animate.next_to(eqns1[1][0][18], direction=DOWN, buff=0.1).scale(0.65).set_color(YELLOW),
            
            #ReplacementTransform(demo_fn1, ),
            
            #run_time=3
        #)
        #temp = VGroup(
         #   MathTex("1").scale(0.75).next_to(ax.c2p(0, 1), direction=LEFT, buff=0.2),
        #    MathTex("3").scale(0.75).next_to(ax.c2p(0, 3), direction=LEFT, buff=0.2)
        #)
        #self.play(FadeIn(temp[0]))
        #self.play(Flash(temp[0]))
        #self.wait()
        #self.play(ReplacementTransform(c, eqns1[1][0][19:]))
        #self.wait()

        self.play(
            Write(eqns1[1][0][8:]),
            ReplacementTransform(demo_fn1, l_plots[0]),
        #    Write(eqns1[1][0][0]),
        #    ReplacementTransform(temp[0], temp[1]),
            run_time=2
        )


        self.play(l_plots[0].animate.fade(0.95))
        for l, eqn in zip(l_plots[1:], [eqns1[-2], eqns2[0], eqns2[-1]]):
            self.play(
                Create(l),
                Write(eqn),
                run_time=2
            )
            self.wait()
            self.play(l.animate.fade(0.8))
        # self.add(l_plots[1:], interp)
        # l_plots.fade(0.8)
        self.play(
            Create(interp),
            *[Write(mob) for mob in [eqns1[0], eqns1[2], eqns1[-1], eqns2[1]]]
        )
        self.wait()
        self.play(ReplacementTransform(eqns, eqn_condensed))
        self.wait(1)
        self.play(*[FadeOut(mobj) for mobj in self.mobjects])

<string>:93: RuntimeWarning: divide by zero encountered in double_scalars                                               


In [61]:
%%manim -ql -v WARNING baricentrica2nd
class baricentrica2nd(Scene):
    def construct(self):
        scale = 0.7
        eqn_blank = MathTex(
            "p(x)=\dfrac{\displaystyle \sum_{i=1}^{n}y_i\dfrac{w_i}{x-x_i}}{\displaystyle \sum_{i=1}^{n}\dfrac{w_i}{(x-x_i)}}"
        ).scale(0.7).to_edge(UP)
        eqn_condensed = MathTex(
            "p(x) = \dfrac{\dfrac{w_1*3}{x-0}+\dfrac{w_2*5}{x-1}+\dfrac{w_3*1}{x-3.5} + \dfrac{w_4*3}{x-5}}{\dfrac{w_1}{x-0}+\dfrac{w_2}{x-1}+\dfrac{w_3}{x-3.5}+\dfrac{w_4}{x-5}}"
        ).scale(0.7).to_edge(UP)
        self.play(Write(eqn_blank))
        self.play(ReplacementTransform(eqn_blank, eqn_condensed))
        tex_title = Tex("Operaciones elementales para evaluar $p(x)$:").scale(0.8).move_to(LEFT*3+UP*1.5)
        self.play(Write(tex_title))
        tex_num = Tex(r"Operaciones en el numerador:").scale(scale).next_to(tex_title, DOWN, aligned_edge = LEFT)
        tex_den = Tex(r"operaciones en el denominador:").scale(scale)
        self.play(Write(tex_num))
        num_group = VGroup(
            Tex(r"$w_i$ multiplicado por $y_i$"), 
            Tex(r"$x$ menos $x_i$"),
            Tex(r"sumar los $n$ terminos")
        ).scale(scale).next_to(tex_num, RIGHT).arrange(DOWN, center = False, aligned_edge = LEFT)
        self.play(Write(num_group[0]))
        self.play(*[Indicate(eqn_condensed[0][idx:idx+4]) for idx in [5,14,23,34]])
        op_group = VGroup(
            MathTex("n"), #mult
            MathTex("n+n"),#restas
            MathTex("2n"), #mult + restas
            MathTex("2n+n"),# + divisiones 
            MathTex("3n"), #mult +restas + divisiones
            MathTex("3n+(n-1)"), # sumas
            MathTex("3n-1"), # todo
        )
        
        
        
        
        
    